In [20]:
#import dependencies
import pandas as pd
import matplotlib as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
import sklearn as skl
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [21]:
#read in the data
#take input from provisional database (csv)
#df = pd.read_csv(Path('./Resources/all_responses_coded.csv'))
# Load the csv file from GitHub
url = 'https://raw.githubusercontent.com/mandymccabe/Final_Project/janet_branch/Data/Final_Project_Full.csv'
url2= 'https://raw.githubusercontent.com/mandymccabe/Final_Project/main/Resources/all_responses_coded.csv'
df = pd.read_csv(url, index_col=0)
df2 = pd.read_csv(url2, index_col=0)
df.head(10)


,political_views,trust_in_press,online_paid_access,impression_with_access_to_data,trump_as_president,age,gender,income,us_region,startdate,...,usa_today,washington_post,fox_news,breitbart,cnn,buzzfeed,huffington_post,time,us_news_and_world_report,other
respondentid,,,,,,,,,,,,,,,,,,,,,
6176264298,Moderate,Fair Amount,No,No Change,Strongly Disapprove,30-44,Male,"$10,000-$24,999",East South Central,2017-05-01,...,1,0,0,0,1,0,0,1,0,0
6176263960,Moderate,Fair Amount,No,Increase Trust,Somewhat Disapprove,18-29,Female,"$0-$9,999",Middle Atlantic,2017-05-01,...,0,0,0,0,1,0,0,0,0,0
6176258621,Liberal,Fair Amount,Yes,Increase Trust,Strongly Disapprove,30-44,Male,"$125,000-$149,999",East North Central,2017-05-01,...,0,1,0,0,0,0,0,0,0,BBC
6176257082,Liberal,Fair Amount,No,Increase Trust,Strongly Disapprove,18-29,Male,"$125,000-$149,999",Mountain,2017-05-01,...,0,1,0,0,0,0,0,0,0,"BBC News, and local news outlets"
6176256111,Liberal,Fair Amount,Yes,No Change,Strongly Disapprove,30-44,Male,"$10,000-$24,999",Pacific,2017-05-01,...,1,0,0,0,1,1,0,1,0,0
6176254380,Liberal,Not Very Much,No,Increase Trust,Strongly Disapprove,30-44,Female,"$200,000 & up",Mountain,2017-05-01,...,0,1,0,0,0,0,0,0,0,0
6176253761,Conservative,Not Very Much,No,Increase Trust,Strongly Approve,30-44,Male,"$50,000-$74,999",East North Central,2017-05-01,...,0,0,0,0,0,0,0,0,0,None
6176248227,Moderate,Not Very Much,No,Increase Trust,Somewhat Disapprove,18-29,Male,"$10,000-$24,999",Pacific,2017-05-01,...,0,0,1,0,1,0,0,0,0,0
6176247712,Moderate,Fair Amount,No,Increase Trust,Strongly Disapprove,30-44,Male,"$25,000-$49,999",West North Central,2017-05-01,...,0,0,0,0,0,0,0,0,0,"NPR, Reuters, New Yorker, Al Jazeera English, ..."


In [22]:
#df2.set_index('RespondentID')
df2.head()

,RespondentID,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A55,A56,A57,A58,A59,A60,A61,A62,StartDate,EndDate
index,,,,,,,,,,,,,,,,,,,,,
0,6176264298,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,5/1/17 15:41,5/1/17 15:43
1,6176263960,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,5/1/17 15:35,5/1/17 15:43
2,6176258621,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,5/1/17 15:38,5/1/17 15:40
3,6176257082,0,0,0,1,0,0,1,0,0,...,1,0,1,0,0,0,0,0,5/1/17 15:38,5/1/17 15:39
4,6176256111,0,0,0,1,0,0,1,0,0,...,0,1,1,0,0,0,0,0,5/1/17 15:34,5/1/17 15:39


In [23]:
PoliticalViews = df.filter(["political_views"], axis=1)
PoliticalViews.head()

,political_views
respondentid,
6176264298,Moderate
6176263960,Moderate
6176258621,Liberal
6176257082,Liberal
6176256111,Liberal


In [24]:
#drop columns we don't want to use to predict and those that do not have an int value

df2 = df2.drop(["A21", "StartDate", "EndDate"], axis=1)
df2.head()
#df2 = df2.join(PoliticalViews)
#df2.head()
##why is this showing as NaN?


,RespondentID,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A53,A54,A55,A56,A57,A58,A59,A60,A61,A62
index,,,,,,,,,,,,,,,,,,,,,
0,6176264298,0,0,1,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
1,6176263960,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,6176258621,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,6176257082,0,0,0,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
4,6176256111,0,0,0,1,0,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0


In [37]:
merged_dfs = pd.merge(df2, PoliticalViews, how='outer', left_on=["RespondentID"], right_on=['respondentid'])
merged_df2 = merged_dfs.drop(['political_views_x'], axis=1)
merged_df2.head()

,RespondentID,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A54,A55,A56,A57,A58,A59,A60,A61,A62,political_views_y
0,6176264298,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,Moderate
1,6176263960,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,Moderate
2,6176258621,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,Liberal
3,6176257082,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,Liberal
4,6176256111,0,0,0,1,0,0,1,0,0,...,0,0,1,1,0,0,0,0,0,Liberal


In [25]:
#df2 = df2.join(PoliticalViews)
#df2.head()

,RespondentID,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A54,A55,A56,A57,A58,A59,A60,A61,A62,political_views
index,,,,,,,,,,,,,,,,,,,,,
0,6176264298,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,NaN
1,6176263960,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,NaN
2,6176258621,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,NaN
3,6176257082,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,NaN
4,6176256111,0,0,0,1,0,0,1,0,0,...,0,0,1,1,0,0,0,0,0,NaN


In [ ]:
#set target and x values
X=df2[list(df2.columns)[5:-1]]
X.head()

In [ ]:
# Create the StandardScaler instance
#scaler = StandardScaler()

# Fit the StandardScaler
#X_scaler = scaler.fit(X_train)

# Scale the data
#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)

In [ ]:
#Y
y=df[list(df.columns)[-1]]
y.head()